In [1]:
from src.dl.loader import BlockRegressDataset
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
log = False

In [2]:
log = True
writer = SummaryWriter('./log')

In [3]:
from src.utils.data import getPandas, writePandas
data = getPandas('data')
test = data.sample(frac=0.2, random_state=10)
data = data.drop(test.index)
validate = data.sample(frac=0.1, random_state=10)
train = data.drop(validate.index)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
writePandas('data_test', test)
writePandas('data_train', train)
writePandas('data_validate', validate)

In [2]:
from src.dl.loader import BlockRegressDataset, RegressDataset
from src.dl.resnet import RegressResNet3d
from sklearn.model_selection import KFold
import collections
train_set = RegressDataset('train')
val_set = RegressDataset('validate')
test_set = RegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)

In [3]:
net = RegressResNet3d().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 0.01
optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=8)
val_loader = DataLoader(val_set, batch_size=8)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
        predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
        y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
        total_loss += loss.item()
        
    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(train_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('train epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    
    if log:
        writer.add_scalar('train/Loss', total_loss, i)
        writer.add_scalar('train/NMSE', nmse, i)
        writer.add_scalar('train/RMSE', rmse, i)
        writer.add_scalar('train/MAE', mae, i)
        writer.add_scalar('train/rsquared', rsquared, i)
        
    net.eval()
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('validation epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('validation/Loss', total_loss, i)
        writer.add_scalar('validation/NMSE', nmse, i)
        writer.add_scalar('validation/RMSE', rmse, i)
        writer.add_scalar('validation/MAE', mae, i)
        writer.add_scalar('validation/rsquared', rsquared, i)

train epoch 0, loss 0.7013780163145757, nmse 0.32174277005611723, rmse 0.23821579993914418, mae 0.19104588079244306, rsquared -0.06789513135618752
validation epoch 0, loss 0.054754057149350555, nmse 0.3517930346327366, rmse 0.19182213765722647, mae 0.15817149899947633, rsquared 0.0409262716685066
train epoch 1, loss 0.6961282372062476, nmse 0.3193726368158819, rmse 0.23733676413791444, mae 0.18991510374005402, rsquared -0.06002843167099692
validation epoch 1, loss 0.05724480492310152, nmse 0.3559610597379185, rmse 0.19295514082510887, mae 0.15361143667357924, rsquared 0.029563217304512945
train epoch 2, loss 0.6924276597135457, nmse 0.31774001310463135, rmse 0.2367293575343636, mae 0.18960644935562554, rsquared -0.05460959689103606
validation epoch 2, loss 0.06874913033622179, nmse 0.4208015431876771, rmse 0.20979434140464173, mae 0.16450759047371666, rsquared -0.14720777611182356
train epoch 3, loss 0.6891127372471969, nmse 0.3162792609948871, rmse 0.23618457051991715, mae 0.189305826

KeyboardInterrupt: 

In [5]:
test_loader = DataLoader(test_set, batch_size=64)
net.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for step, [img, labels, score] in enumerate(test_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        #score = torch.Tensor(np.random.randint(0, 3, score.size())).type(torch.LongTensor).cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)
        
        predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
        y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
        total_loss += loss.item()
nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
mse = np.sum((y - predy) ** 2) / len(test_set)
rmse = np.sqrt(mse)
mae = np.sum(np.abs(y - predy)) / len(test_set)
meany = np.mean(y)
rsquared = 1 - np.sum((predy - y) ** 2) / np.sum((y - meany) ** 2)
print('epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))

epoch 99, loss 11.664599132818774, nmse 0.2849210541321967, rmse 0.4760008485561512, mae 0.8189351787140201, rsquared -0.016067644564553785


In [ ]:
loss_fn = nn.SmoothL1Loss().cuda()
lr = 1e-3
optim = torch.optim.SGD([
    {'params': net.fc.weight, 'lr': 1e-2}
    ], lr=lr, momentum=0.3)
#optim = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.99)
#optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.3, 0.1])
epoch = 100
dataset = BlockRegressDataset('train')
kf = KFold(n_splits=5, shuffle=True, random_state=10)
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels).squeeze(-1)
                loss = loss_fn(output.float(), score.float())
        
                predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
                y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

                total_loss += loss.item()

        nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
        mse = np.sum((y - predy) ** 2) / len(train_idx)
        mae = np.sum(np.abs(y - predy)) / len(train_idx)
        meany = np.mean(y)
        rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
        print('epoch {}, loss {}, nmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, mae, rsquared))
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

writer.close()

In [ ]:
predy = []
y = []
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    net.eval()
    with torch.no_grad():
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())
    
            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_idx)
    mae = np.sum(np.abs(y - predy)) / len(train_idx)
    meany = np.mean(y)
    rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
    print(rsquared)
    print(mse)
    print(mae)
    break